# Ex1) P2P Task

In [1]:
from tasho.templates.P2P import P2P
from tasho import TaskModel, default_mpc_options
from tasho.templates.Regularization import Regularization
from tasho.Variable import Variable
from tasho.ConstraintExpression import ConstraintExpression
from tasho.Expression import Expression
from tasho.MPC import MPC 
from tasho import environment as env
from tasho import WorldSimulator
from tasho.OCPGenerator import OCPGenerator
from robotshyu import Robot as rob
from time import sleep
import numpy as np
import casadi as cs
import pybullet as p
import tasho
import robotshyu
import os

pybullet build time: May 20 2022 19:44:17


In [2]:
# Options for example codes
frame_enable = True

In [3]:
robot = rob.Robot("mmo_500_ppr")
link_name = 9
goal_pose = Variable(robot.name, "goal_pose", "magic_number", (4,4), np.array(
        [[1, 0, 0, 1], [0, 0, 0, 0.0], [0, 0, 1, 0.7], [0, 0, 0, 1]]
    ))
q0 = [-1.0193752249977548, -0.05311582280659044, 0.0,
    1.3191046402052224, 0.0, 0.0, 0.0, 0.0, 0.0]
robot.set_joint_acceleration_limits(lb=-360*3.14159/180, ub=360*3.14159/180)
ndof = robot.ndof
q_current = Variable(robot.name, "jointpos_init", 'magic_number', (ndof, 1), q0)

Loading robot params from json ...
Loaded 9-DoF robot: mmo_500_ppr


In [4]:
task_P2P = P2P(robot, link_name, goal_pose, q_current, 0.001)

In [5]:
task_P2P.write_task_graph("MMO500_P2P_Task.svg")

In [6]:
task_P2P.add_path_constraints(Regularization(task_P2P.variables['qd_'+robot.name], 1),
                              Regularization(task_P2P.variables['qdd_'+robot.name], 0.1))

In [7]:
task_P2P.write_task_graph("MMO500_P2P_Task2.svg")

In [8]:
horizon_steps = 10
horizon_period = 3.0
q_pred = [0]*horizon_steps

OCP_gen = OCPGenerator(task_P2P, False, {"time_period": horizon_period, "horizon_steps":horizon_steps})
q_ocp = OCP_gen.stage_tasks[0].variables['q_'+robot.name].x
OCP_gen.tc.set_initial(q_ocp, q0)
OCP_gen.tc.set_ocp_solver(
    "ipopt", 
    # {"ipopt":{"linear_solver":"ma27"}}
)
OCP_gen.tc.solve_ocp()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.7, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:      675
Number of nonzeros in inequality constraint Jacobian.:      277
Number of nonzeros in Lagrangian Hessian.............:      208

Total number of variables............................:      288
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      210
Total number of inequality constrai

In [9]:
t_grid, qsol = OCP_gen.tc.sol_sample(q_ocp)
_, q_dot_sol = OCP_gen.tc.sol_sample(OCP_gen.stage_tasks[0].variables['qd_'+robot.name].x)

obj = WorldSimulator.WorldSimulator()

    # Add robot to the world environment
position = [0.0, 0.0, 0.0]
orientation = [0.0, 0.0, 0.0, 1.0]

robotID = obj.add_robot(position,orientation,robot_pkg=robot)
frameID0 = obj.add_robot(position,orientation,robot_pkg=robot,frame=True)
robotIDs=[robotID,frameID0]

if frame_enable:
    frameIDs = [0]*horizon_steps
    for i in range(horizon_steps):
        frameIDs[i] = obj.add_robot(position,orientation,robot_pkg=robot,frame=True)

for i in range(horizon_steps):
    q_pred[i]=qsol[i]

joint_indices = [0, 1, 2, 13, 14, 15, 16, 17, 18]

obj.resetMultiJointState(robotIDs, joint_indices, [q0])

if frame_enable:
    obj.resetMultiJointState(frameIDs, joint_indices, [q0])
    obj.resetMultiJointState(frameIDs,joint_indices,q_pred)

In [10]:
for i in range(horizon_steps + 1):
    sleep(horizon_period*0.5/horizon_steps)
    obj.resetMultiJointState(
        robotIDs, joint_indices, [qsol[i]]
    )

sleep(0.5)
obj.end_simulation()

Ending simulation


# Ex2) Object picking MPC

In [11]:
robot = rob.Robot("mmo_500_ppr")
link_name = 9
robot.set_joint_acceleration_limits(lb=-360*3.14159/180, ub=360*3.14159/180)

# Define initial conditions of the robot
q0_val = [-1.0, -0.523598, 1.5709, 3.14, 0, -0.523598, 0.0, 0.0, 0.0]
qd0_val = [0] * robot.ndof

################################################
# Task spacification - Approximation to object
################################################

# Select prediction horizon and sample time for the MPC execution
horizon_size = 10
t_mpc = 0.05

q_init = Variable(robot.name, 'q_init', 'parameter', (robot.ndof,1))
qd_init = Variable(robot.name, 'qd_init', 'parameter', (robot.ndof,1))
goal_pose = Variable(robot.name, "goal_pose", 'parameter', (4,4))

task_P2P = P2P(robot, link_name, goal_pose, q_init, 0.001)
task_P2P.write_task_graph("MMO500_P2P_Task4.svg")

Loading robot params from json ...
Loaded 9-DoF robot: mmo_500_ppr


In [12]:
task_P2P.remove_initial_constraints(task_P2P.constraint_expressions['stationary_qd_mmo_500_ppr'])
task_P2P.remove_terminal_constraints('rot_con_pose_9_mmo_500_ppr_vs_goal',
                                    'trans_con_pose_9_mmo_500_ppr_vs_goal')
task_P2P.write_task_graph("MMO500_P2P_Task5.svg")

In [13]:
task_P2P.add_initial_constraints(ConstraintExpression(robot.name, "eq", Expression(robot.name, "err_qd_qinit", lambda a, b : a - b, qd_init, task_P2P.variables['qd_mmo_500_ppr']),
                             "hard", reference = 0))
task_P2P.add_path_constraints(Regularization(task_P2P.expressions['trans_error_pose_9_mmo_500_ppr_vs_goal'], 2e0, norm = "L1"),
                            Regularization(task_P2P.expressions['ax_ang_error_pose_9_mmo_500_ppr_vs_goal'], 8e-1), 
                            Regularization(task_P2P.variables['qd_mmo_500_ppr'], 3e-2),
                            Regularization(task_P2P.variables['qdd_mmo_500_ppr'],4e-2))

task_P2P.write_task_graph("MMO500_P2P_Task6.svg")

In [14]:
pOCP = OCPGenerator(task_P2P, False, {"time_period":horizon_size*t_mpc, "horizon_steps":horizon_size})

tc = pOCP.tc

################################################
# Set solver and discretization options
################################################
tc.set_ocp_solver("ipopt", {"ipopt": {"print_level": 5,"tol": 1e-3}})
# tc.set_ocp_solver("ipopt", {"ipopt": {"print_level": 0,"tol": 1e-3, "linear_solver":"ma27"}}) #use this if you have hsl


################################################
# Set parameter values
################################################
q = pOCP.stage_tasks[0].variables['q_mmo_500_ppr'].x
qd = pOCP.stage_tasks[0].variables['qd_mmo_500_ppr'].x
q_0 = pOCP.stage_tasks[0].variables['q_init_mmo_500_ppr'].x
qd_0 = pOCP.stage_tasks[0].variables['qd_init_mmo_500_ppr'].x
goal_pose = pOCP.stage_tasks[0].variables['goal_pose_mmo_500_ppr'].x
goal_pose_val =  cs.vertcat(
    cs.hcat([0, 1, 0, 0.5]),
    cs.hcat([1, 0, 0, 0.0]),
    cs.hcat([0, 0, -1, 0.25]),
    cs.hcat([0, 0, 0, 1]),
)
tc.set_initial(q, q0_val)
tc.set_value(goal_pose, goal_pose_val)
tc.set_value(q_0, q0_val)
tc.set_value(qd_0, qd0_val)

# Add an output port for joint velocities as well
tc.tc_dict["out_ports"].append({"name":"port_out_qd_mmo_500_ppr", "var":"qd_mmo_500_ppr", "desc": "output port for the joint velocities"})

# Add a monitor for termination criteria
tc.add_monitor({"name":"termination_criteria", "expression":cs.sqrt(cs.sumsqr(pOCP.stage_tasks[0].expressions["trans_error_pose_9_mmo_500_ppr_vs_goal"].x)) - 2e-2, "reference":0, "lower":True, "initial":True})

mpc_options = default_mpc_options.get_default_mpc_options()


tc.ocp_solver = "ipopt"
tc.ocp_options = mpc_options["ipopt_hsl"]["options"]
tc.mpc_solver = tc.ocp_solver
tc.mpc_options = tc.ocp_options
tc.set_ocp_solver("ipopt", tc.mpc_options)
os.system("export OMP_NUM_THREADS = 1")

sol = tc.solve_ocp()

      solver startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa/X.org
GL_RENDERER=llvmpipe (LLVM 12.0.0, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 21.2.6
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 21.2.6
Vendor = Mesa/X.org
Renderer = llvmpipe (LLVM 12.0.0, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Mesa/X.org
ven = Mesa/X.org
numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main sema

sh: 1: export: : bad variable name


In [15]:
dir_casadi_func = "casadi_dir"
os.makedirs("./"+dir_casadi_func, exist_ok=True)
vars_db = tc.generate_MPC_component("./"+dir_casadi_func+"/", {"ocp_cg_opts":{"save":True, "codegen":False, "jit":False}, "mpc":True, "mpc_cg_opts":{"save":True, "codegen":False, "jit":False}})

@1=(opti1_p_2[12:15]-fk_T(opti1_x_1){9}[12:15]), (sqrt(dot(@1, @1))-0.02)
@1=(opti1_p_2[12:15]-fk_T(opti1_x_1){9}[12:15]), (sqrt(dot(@1, @1))-0.02)


In [16]:
MPC_component = MPC("mmo_500_ppr_obj_pickup", "./"+dir_casadi_func+"/"+ tc.name + ".json")

In [17]:
obj = WorldSimulator.WorldSimulator(bullet_gui=True)

# Add robot to the world environment
position = [0.0, 0.0, 0.0]
orientation = [0.0, 0.0, 0.0, 1.0]
robotID = obj.add_robot(position,orientation,robot_pkg=robot)

if frame_enable:
    frameIDs = [0]*horizon_size
    for i in range(horizon_size):
        frameIDs[i] = obj.add_robot(position,orientation,robot_pkg=robot,frame=True)

# Set environment
environment = env.Environment()
package_path = tasho.__path__[0]
cube1 = env.Cube(length = 1, position = [0.5, -0.2, 0.35], orientation = [0.0, 0.0, 0.0, 1.0], urdf = package_path+"/models/objects/cube_small.urdf")
environment.add_object(cube1, "cube")
table1 = env.Box(height = 0.3, position = [0.5, 0, 0], orientation = [0.0, 0.0, 0.7071080798594737, 0.7071054825112364], urdf =package_path+ "/models/objects/table.urdf")
environment.add_object(table1, "table1")

cube2 = env.Cube(length = 1, position = [0.5, -0.2, 0.35], orientation = [0.0, 0.0, 0.0, 1.0], urdf = package_path+"/models/objects/cube_small_col.urdf", fixed = True)
environment.add_object(cube2, "cube2")
environment.set_in_world_simulator(obj)
cubeID = environment.get_object_ID("cube")
cube2ID = environment.get_object_ID("cube2")

#p.resetBaseVelocity(cubeID, linearVelocity=[0, 0.8, 0])
# Determine number of samples that the simulation should be executed
no_samples = int(t_mpc / obj.physics_ts)
if no_samples != t_mpc / obj.physics_ts:
    print("[ERROR] MPC sampling time not integer multiple of physics sampling time")

# Correspondence between joint numbers in bullet and OCP
joint_indices = [0, 1, 2, 13, 14, 15, 16, 17, 18]

 # Begin the visualization by applying the initial control signal
ts, q_sol = tc.sol_sample(q, grid="control")
ts, q_dot_sol = tc.sol_sample(qd, grid="control")

obj.resetJointState(robotID,joint_indices,q0_val)
if frame_enable==True:
    obj.resetMultiJointState(frameIDs, joint_indices, [q0_val])

obj.setController(
    robotID, "velocity", joint_indices, targetVelocities=q_dot_sol[0]
)

q_log = []
q_dot_log = []
predicted_pos_log = []
q_pred = [0]*horizon_size

In [ ]:
# Execute the MPC loop

p.resetBaseVelocity(cubeID, linearVelocity=[0, 1, 0])

for i in range(horizon_size * 100):
    print("----------- MPC execution -----------")

    q_now = obj.readJointPositions(robotID, joint_indices)
    qd_now = obj.readJointVelocities(robotID, joint_indices)
        
    MPC_component.input_ports["port_inp_q_init_mmo_500_ppr"]["val"] = q_now
    MPC_component.input_ports["port_inp_qd_init_mmo_500_ppr"]["val"] = qd_now

    # Predict the position of the target object (cube)
    lin_vel, ang_vel = p.getBaseVelocity(cubeID)
    lin_vel = cs.DM(lin_vel)
    lin_pos, _ = p.getBasePositionAndOrientation(cubeID)
    lin_pos = cs.DM(lin_pos)
    time_to_stop = cs.norm_1(lin_vel) / 0.3
    predicted_pos = (
        cs.DM(lin_pos)
        + cs.DM(lin_vel) * time_to_stop
        - 0.5 * 0.5 * lin_vel / (cs.norm_1(lin_vel) + 1e-3) * time_to_stop ** 2
    )
    predicted_pos_log.append(predicted_pos.full())
    p.resetBasePositionAndOrientation(cube2ID, predicted_pos.full(), [0.0, 0.0, 0.0, 1.0])
    predicted_pos[2] += 0.06  # cube height
    print("Predicted position of cube", predicted_pos)
    predicted_pos_val = cs.vertcat(
        cs.hcat([0, 0, 1, predicted_pos[0]]),
        cs.hcat([0, 1, 0, predicted_pos[1]]),
        cs.hcat([-1, 0, 0, predicted_pos[2]]),
        cs.hcat([0, 0, 0, 1]),
    )

    MPC_component.input_ports["port_inp_goal_pose_mmo_500_ppr"]["val"] = cs.vec(predicted_pos_val)
        
    if i == 0:
        MPC_component.configMPC()

    MPC_component.runMPC()
    if frame_enable==True:
        sol = MPC_component.res_vals
        for i in range(horizon_size):
            q_pred[i]=sol[robot.ndof*i:robot.ndof*i+robot.ndof].full()
        obj.resetMultiJointState(frameIDs, joint_indices, q_pred)

    q_log.append(q_now)
    q_dot_log.append(qd_now)

    # Set control signal to the simulated robot
    qd_control_sig = MPC_component.output_ports["port_out_qd_mmo_500_ppr"]["val"].full()
    qdd_control_sig = (MPC_component.output_ports["port_out_qdd_mmo_500_ppr"]["val"] * t_mpc).full()
    obj.setController(
        robotID, "velocity", joint_indices, targetVelocities=qd_control_sig + qdd_control_sig
    )

    # Simulate
    obj.run_simulation(no_samples)
#     sleep(0.1)

    # Termination criteria
#     if "termination_criteria_true" in MPC_component.event_output_port:
#         break

----------- MPC execution -----------
Predicted position of cube [0.5, 0.35833055833056, 0.41]
startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa/X.org
GL_RENDERER=llvmpipe (LLVM 12.0.0, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 21.2.6
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 21.2.6
Vendor = Mesa/X.org
Renderer = llvmpipe (LLVM 12.0.0, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Mesa/X.org
ven = Mesa/X.org
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       

----------- MPC execution -----------
Predicted position of cube [0.50000036490589, 0.95173551892937, -0.58961182386647]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  94.00us ( 23.50us)  93.63us ( 23.41us)         4
       nlp_g  |  46.00us ( 11.50us)  47.68us ( 11.92us)         4
    nlp_grad  |  96.00us ( 96.00us)  96.12us ( 96.12us)         1
  nlp_grad_f  | 226.00us ( 56.50us) 225.50us ( 56.38us)         4
  nlp_hess_l  |   1.01ms (337.00us)   1.01ms (337.30us)         3
   nlp_jac_g  |  80.00us ( 20.00us)  79.39us ( 19.85us)         4
       total  |   9.54ms (  9.54ms)   3.73ms (  3.73ms)         1
----------- MPC execution -----------
Predicted position of cube [0.50000045167558, 1.1746415190892, -1.4583466437659]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  97.00us ( 24.25us)  97.44us ( 24.36us)         4
       nlp_g  |  39.00us (  9.75us)  40.65us ( 10.16us)         4
    nlp_grad  |  95.00us ( 95.00us

----------- MPC execution -----------
Predicted position of cube [0.49989914972401, 0.84595331847541, 0.084989939918656]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 142.00us ( 35.50us) 141.96us ( 35.49us)         4
       nlp_g  |  69.00us ( 17.25us)  69.75us ( 17.44us)         4
    nlp_grad  | 161.00us (161.00us) 160.81us (160.81us)         1
  nlp_grad_f  | 346.00us ( 86.50us) 347.35us ( 86.84us)         4
  nlp_hess_l  |   1.53ms (510.67us)   1.53ms (510.34us)         3
   nlp_jac_g  | 120.00us ( 30.00us) 120.69us ( 30.17us)         4
       total  |  21.48ms ( 21.48ms)   5.47ms (  5.47ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49989738299818, 0.85430652628286, 0.084989902026272]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 750.00us (187.50us)  93.85us ( 23.46us)         4
       nlp_g  |  49.00us ( 12.25us)  48.32us ( 12.08us)         4
    nlp_grad  |  94.00us ( 94.00

----------- MPC execution -----------
Predicted position of cube [0.49988452648185, 0.91569448659217, 0.08498976999669]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  92.00us ( 23.00us)  92.76us ( 23.19us)         4
       nlp_g  |  48.00us ( 12.00us)  48.42us ( 12.11us)         4
    nlp_grad  |  96.00us ( 96.00us)  95.97us ( 95.97us)         1
  nlp_grad_f  | 221.00us ( 55.25us) 221.08us ( 55.27us)         4
  nlp_hess_l  |   1.00ms (333.33us)   1.00ms (333.37us)         3
   nlp_jac_g  |  78.00us ( 19.50us)  78.21us ( 19.55us)         4
       total  |  11.06ms ( 11.06ms)   3.74ms (  3.74ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49988449085664, 0.91599072816808, 0.084989546156084]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  96.00us ( 24.00us)  96.51us ( 24.13us)         4
       nlp_g  |  51.00us ( 12.75us)  49.18us ( 12.29us)         4
    nlp_grad  |  96.00us ( 96.00u

----------- MPC execution -----------
Predicted position of cube [0.49988465462319, 0.91597876901318, 0.084989989602974]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  91.00us ( 22.75us)  90.96us ( 22.74us)         4
       nlp_g  |  47.00us ( 11.75us)  47.39us ( 11.85us)         4
    nlp_grad  |  97.00us ( 97.00us)  97.34us ( 97.34us)         1
  nlp_grad_f  |   1.27ms (317.50us) 227.08us ( 56.77us)         4
  nlp_hess_l  |  13.38ms (  4.46ms)   1.03ms (343.09us)         3
   nlp_jac_g  |  78.00us ( 19.50us)  78.26us ( 19.57us)         4
       total  |  23.93ms ( 23.93ms)   3.89ms (  3.89ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49988465627458, 0.91597877211122, 0.084989989725133]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   1.04ms (259.25us)  97.85us ( 24.46us)         4
       nlp_g  |   4.04ms (  1.01ms)  44.06us ( 11.02us)         4
    nlp_grad  | 438.00us (438.00

...

----------- MPC execution -----------
Predicted position of cube [0.49988481024165, 0.91597886374106, 0.084989991124243]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 136.00us ( 34.00us)  90.85us ( 22.71us)         4
       nlp_g  |  49.00us ( 12.25us)  47.16us ( 11.79us)         4
    nlp_grad  |  95.00us ( 95.00us)  94.58us ( 94.58us)         1
  nlp_grad_f  | 225.00us ( 56.25us) 224.73us ( 56.18us)         4
  nlp_hess_l  |   2.27ms (758.33us)   1.02ms (340.45us)         3
   nlp_jac_g  | 104.00us ( 26.00us)  81.60us ( 20.40us)         4
       total  |  11.52ms ( 11.52ms)   3.98ms (  3.98ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49988481024165, 0.91597886374106, 0.084989991124243]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  86.00us ( 21.50us)  86.00us ( 21.50us)         4
       nlp_g  |  46.00us ( 11.50us)  47.06us ( 11.77us)         4
    nlp_grad  | 185.00us (185.00

----------- MPC execution -----------
Predicted position of cube [0.49988481024165, 0.91597886374106, 0.084989991124243]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  94.00us ( 23.50us)  94.50us ( 23.62us)         4
       nlp_g  |  56.00us ( 14.00us)  55.94us ( 13.99us)         4
    nlp_grad  |  96.00us ( 96.00us)  95.94us ( 95.94us)         1
  nlp_grad_f  | 231.00us ( 57.75us) 233.37us ( 58.34us)         4
  nlp_hess_l  |   8.98ms (  2.99ms)   1.08ms (361.33us)         3
   nlp_jac_g  |  76.00us ( 19.00us)  75.61us ( 18.90us)         4
       total  |  12.31ms ( 12.31ms)   4.41ms (  4.41ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49988481024165, 0.91597886374106, 0.084989991124243]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  92.00us ( 23.00us)  92.16us ( 23.04us)         4
       nlp_g  |  48.00us ( 12.00us)  47.62us ( 11.91us)         4
    nlp_grad  |  96.00us ( 96.00

----------- MPC execution -----------
Predicted position of cube [0.49988481024165, 0.91597886374106, 0.084989991124243]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  91.00us ( 22.75us)  89.96us ( 22.49us)         4
       nlp_g  |  48.00us ( 12.00us)  46.87us ( 11.72us)         4
    nlp_grad  | 100.00us (100.00us) 100.31us (100.31us)         1
  nlp_grad_f  | 232.00us ( 58.00us) 232.89us ( 58.22us)         4
  nlp_hess_l  |   1.07ms (356.67us)   1.08ms (358.57us)         3
   nlp_jac_g  |  82.00us ( 20.50us)  83.48us ( 20.87us)         4
       total  |  11.96ms ( 11.96ms)   3.96ms (  3.96ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49988481024165, 0.91597886374106, 0.084989991124243]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  85.00us ( 21.25us)  86.91us ( 21.73us)         4
       nlp_g  |  47.00us ( 11.75us)  47.16us ( 11.79us)         4
    nlp_grad  |  93.00us ( 93.00

----------- MPC execution -----------
Predicted position of cube [0.49988481024165, 0.91597886374106, 0.084989991124243]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   1.91ms (477.75us) 117.47us ( 29.37us)         4
       nlp_g  |   1.43ms (358.00us)  83.80us ( 20.95us)         4
    nlp_grad  | 481.00us (481.00us) 110.55us (110.55us)         1
  nlp_grad_f  |  17.90ms (  4.48ms) 254.25us ( 63.56us)         4
  nlp_hess_l  |   4.33ms (  1.44ms)   1.02ms (340.66us)         3
   nlp_jac_g  | 210.00us ( 52.50us)  81.17us ( 20.29us)         4
       total  |  46.88ms ( 46.88ms)   4.56ms (  4.56ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49988481024165, 0.91597886374106, 0.084989991124243]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  83.00us ( 20.75us)  83.91us ( 20.98us)         4
       nlp_g  |  46.00us ( 11.50us)  45.70us ( 11.43us)         4
    nlp_grad  |  91.00us ( 91.00

----------- MPC execution -----------
Predicted position of cube [0.49988481024165, 0.91597886374106, 0.084989991124243]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  82.00us ( 20.50us)  83.49us ( 20.87us)         4
       nlp_g  |  47.00us ( 11.75us)  47.35us ( 11.84us)         4
    nlp_grad  |  90.00us ( 90.00us)  89.70us ( 89.70us)         1
  nlp_grad_f  | 208.00us ( 52.00us) 208.61us ( 52.15us)         4
  nlp_hess_l  |   9.00ms (  3.00ms)   1.00ms (334.22us)         3
   nlp_jac_g  |  78.00us ( 19.50us)  77.93us ( 19.48us)         4
       total  |  11.63ms ( 11.63ms)   3.63ms (  3.63ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49988481024165, 0.91597886374106, 0.084989991124243]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |  88.00us ( 22.00us)  88.31us ( 22.08us)         4
       nlp_g  |  48.00us ( 12.00us)  47.96us ( 11.99us)         4
    nlp_grad  |  93.00us ( 93.00

In [ ]:
obj.end_simulation()

max(MPC_component.solver_times)

# Ex3) Nonholonomic Constraint